In [30]:
import pandas as pd
import numpy as np
import quandl
import FundamentalAnalysis as fa
import datetime as dt
import pandas as pd
import pandas_datareader.data as web
import matplotlib.dates as mdates
import urllib
import ta
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import train_test_split

quandl.ApiConfig.api_key = 'exUkGQLmC9_T8J1TU3Xz'
fm = '07a2e7ec7eaf928f2a35b6e904281ba5'

In [31]:
#CONSUMER SENTIMENT INDEX
df_monthly_sentiment = quandl.get('UMICH/SOC1')
#df_monthly_sentiment.tail()

#NATIONAL GDP DATA
df_gdp = quandl.get('FRED/GDP')
df_gdp.tail()

,Value
Date,
2019-01-01,21098.827
2019-04-01,21340.267
2019-07-01,21542.540
2019-10-01,21729.124
2020-01-01,21537.940


In [32]:
def get_price_data(ticker):
    start = dt.datetime(2010, 4, 5)
    end = dt.datetime.now()
    df = web.DataReader(ticker, 'yahoo', start, end)
    df.reset_index(inplace=True)
    df.set_index("Date", inplace=True)
    return df

df_tsla = get_price_data('TSLA')
df_tsla

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-06-29,25.000000,17.540001,19.000000,23.889999,18766300,23.889999
2010-06-30,30.420000,23.299999,25.790001,23.830000,17187100,23.830000
2010-07-01,25.920000,20.270000,25.000000,21.959999,8218800,21.959999
2010-07-02,23.100000,18.709999,23.000000,19.200001,5139800,19.200001
2010-07-06,20.000000,15.830000,20.000000,16.110001,6866900,16.110001
...,...,...,...,...,...,...
2020-05-15,805.049988,786.549988,790.349976,799.169983,10518400,799.169983
2020-05-18,834.719971,803.880005,827.780029,813.630005,11698100,813.630005
2020-05-19,822.070007,806.080017,815.169983,808.010010,9636500,808.010010


In [33]:
def get_weekly_returns(price_data):
    df_ohlc = price_data['Adj Close'].resample('7D').ohlc()
    df_ohlc = df_ohlc.reset_index()
    #df_ohlc['Date'] = df_ohlc['Date'].map(mdates.date2num)
    df_ohlc['future_close_pct_change'] = df_ohlc['close'].pct_change()
    df_ohlc['current_close_pct_change'] = df_ohlc['close'].pct_change()
    df_ohlc['future_close_pct_change'] = df_ohlc['future_close_pct_change'].shift(periods=-1)
    num_rows = df_ohlc.shape[0]
    df_ohlc = df_ohlc.drop(num_rows-1)
    df_ohlc = df_ohlc.drop(0)
    return df_ohlc
    
def get_50_day_ma(price_data):
    series = price_data['Adj Close'].rolling(window=50,min_periods=0).mean()
    return series

weekly_ma50 = get_50_day_ma(df_tsla).resample('7D').mean().to_frame()
df_weekly = get_weekly_returns(df_tsla)
#weekly_ma50
df_weekly

,Date,open,high,low,close,future_close_pct_change,current_close_pct_change
1,2010-07-06,16.110001,17.459999,15.800000,17.049999,0.285044,-0.111979
2,2010-07-13,18.139999,21.910000,18.139999,21.910000,-0.043816,0.285044
3,2010-07-20,20.299999,21.290001,20.219999,20.950001,-0.001432,-0.043816
4,2010-07-27,20.549999,20.920000,19.940001,20.920000,-0.063097,-0.001432
5,2010-08-03,21.950001,21.950001,19.590000,19.600000,-0.041837,-0.063097
...,...,...,...,...,...,...,...
511,2020-04-14,709.890015,753.890015,709.890015,746.359985,0.070194,0.146570
512,2020-04-21,686.719971,798.750000,686.719971,798.750000,-0.047023,0.070194
513,2020-04-28,769.119995,800.510010,701.320007,761.190002,0.065818,-0.047023
514,2020-05-05,768.210022,819.419983,768.210022,811.289978,0.002884,0.065818


In [48]:
ticker = 'AAPL'

# Show the available companies
companies = fa.available_companies()

# Collect general company information
profile = fa.profile(ticker)

# Collect recent company quotes
quotes = fa.quote(ticker)

# Collect market cap and enterprise value
enterprise_value = fa.enterprise(ticker)

# Show recommendations of Analysts
ratings = fa.rating(ticker)

# Obtain DCFs over time
dcf_quarterly = fa.discounted_cash_flow(ticker, period="quarter")

# Collect the Balance Sheet statements
balance_sheet_quarterly = fa.balance_sheet_statement(ticker, period="quarter")

# Collect the Income Statements
income_statement_quarterly = fa.income_statement(ticker, period="quarter")

# Collect the Cash Flow Statements
cash_flow_statement_quarterly = fa.cash_flow_statement(ticker, period="quarter")

# Show Key Metrics
key_metrics_quarterly = fa.key_metrics(ticker, period="quarter")

# Show a large set of in-depth ratios
financial_ratios = fa.financial_ratios(ticker)

# Show the growth of the company
growth_quarterly = fa.financial_statement_growth(ticker, period="quarter")

KeyError: 'symbolsList'

In [49]:
# THESE ARE THE TREND INDICATORS

adx = ta.trend.ADXIndicator(high = df_tsla['High'], low = df_tsla['Low'], close = df_tsla['Close']).adx()
#adx_neg = adx_obj.adx_neg()
#adx_pos = adx_obj.adx_pos()
aroon = ta.trend.AroonIndicator(close = df_tsla['Close']).aroon_indicator()
macd = ta.trend.MACD(close = df_tsla['Close']).macd()

# THESE ARE MOMENTUM INDICATORS

rsi = ta.momentum.RSIIndicator(close = df_tsla['Close']).rsi()
stoch = ta.momentum.StochasticOscillator(high = df_tsla['High'], low = df_tsla['Low'],
                                             close = df_tsla['Close']).stoch()

#THESE ARE VOLUME INDICATORS

adl = ta.volume.AccDistIndexIndicator(high = df_tsla['High'], low = df_tsla['Low'],
                                            close = df_tsla['Close'], volume=df_tsla['Volume']).acc_dist_index()
obv = ta.volume.OnBalanceVolumeIndicator(close = df_tsla['Close'], volume = df_tsla['Volume']).on_balance_volume()

#THESE ARE VOLATILITY INDICATORS

mbol = ta.volatility.BollingerBands(close = df_tsla['Close']).bollinger_mavg()

C:\Users\nrdas\Anaconda3\envs\finance\lib\site-packages\ta\trend.py:567: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i]/self._trs[i])
C:\Users\nrdas\Anaconda3\envs\finance\lib\site-packages\ta\trend.py:571: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i]/self._trs[i])


In [50]:
adx_weekly = adx.rolling(window=5,min_periods=0).mean().resample('7D').mean().to_frame()
aroon_weekly = aroon.rolling(window=5,min_periods=0).mean().resample('7D').mean().to_frame()
macd_weekly = macd.rolling(window=5,min_periods=0).mean().resample('7D').mean().to_frame()
rsi_weekly = rsi.rolling(window=5,min_periods=0).mean().resample('7D').mean().to_frame()
stoch_weekly = stoch.rolling(window=5,min_periods=0).mean().resample('7D').mean().to_frame()
obv_weekly = obv.rolling(window=5,min_periods=0).mean().resample('7D').mean().to_frame()
adl_weekly = adl.rolling(window=5,min_periods=0).mean().resample('7D').mean().to_frame()
mbol_weekly = mbol.rolling(window=5,min_periods=0).mean().resample('7D').mean().to_frame()

In [51]:
df_weekly['adx'] = adx_weekly[1:-1].set_index(df_weekly.index)
df_weekly['aroon'] = aroon_weekly[1:-1].set_index(df_weekly.index)
df_weekly['macd'] = macd_weekly[1:-1].set_index(df_weekly.index)
df_weekly['rsi'] = rsi_weekly[1:-1].set_index(df_weekly.index)
df_weekly['stoch'] = stoch_weekly[1:-1].set_index(df_weekly.index)
df_weekly['obv'] = obv_weekly[1:-1].set_index(df_weekly.index)
df_weekly['adl'] = adl_weekly[1:-1].set_index(df_weekly.index)
df_weekly['mbol'] = mbol_weekly[1:-1].set_index(df_weekly.index)
df_weekly['ma_50'] = weekly_ma50[1:-1].set_index(df_weekly.index)

In [52]:
#df_weekly = df_weekly[['Date', 'open', 'high', 'low', 'close', 'adx', 'aroon', 'macd', 'rsi', 'stoch', 'obv', 'future_close_pct_change']]
df_weekly = df_weekly.drop(columns=['Date'])[5:]
df_weekly

,open,high,low,close,future_close_pct_change,current_close_pct_change,adx,aroon,macd,rsi,stoch,obv,adl,mbol,ma_50
6,19.030001,19.030001,17.600000,18.780001,0.071885,-0.041837,5.495650,56.80,-0.392014,40.530536,11.280591,-12892920.0,-5.692118e+06,20.366020,19.916169
7,19.150000,20.129999,18.770000,20.129999,-0.012916,0.071885,7.022581,-5.92,-0.633730,42.509568,27.979394,-13229484.0,-6.031441e+06,19.987260,19.756200
8,19.200001,19.900000,19.200001,19.870001,0.059386,-0.012916,6.733018,-28.00,-0.410003,48.677272,57.922206,-12143620.0,-5.388070e+06,19.604760,19.728598
9,19.480000,21.059999,19.480000,21.049999,-0.015677,0.059386,7.629018,-28.00,-0.153643,51.982231,81.853666,-11793255.0,-5.114440e+06,19.395375,19.758711
10,20.540001,20.900000,20.170000,20.719999,0.016409,-0.015677,10.788441,-6.24,0.125415,55.521533,82.125034,-11531872.0,-4.844251e+06,19.481480,19.756847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,709.890015,753.890015,709.890015,746.359985,0.070194,0.146570,19.428062,48.80,3.382279,62.542783,93.756636,599453580.0,4.408348e+08,514.452863,645.880801
512,686.719971,798.750000,686.719971,798.750000,-0.047023,0.070194,22.981946,82.24,40.314836,63.591478,85.550751,640405704.0,4.433454e+08,583.455443,641.697159
513,769.119995,800.510010,701.320007,761.190002,0.065818,-0.047023,23.936563,75.52,50.565948,62.176529,79.795491,642525832.0,4.536785e+08,644.769324,635.085919
514,768.210022,819.419983,768.210022,811.289978,0.002884,0.065818,25.033594,74.72,48.594576,59.051575,52.021025,633968468.0,4.447785e+08,710.521405,629.278561


In [53]:
#WE NEED TO GENERATE TEHSE DATAFRAMES FOR ALL STOCKS, APPEND THEM ALL, THEN RUN THIS CODE

features = df_weekly[['open', 'high', 'low', 'close', 'adx', 'aroon', 'macd', 'rsi', 'stoch', 'obv', 'ma_50', 'current_close_pct_change']].values
labels = df_weekly['future_close_pct_change'].values
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
reg1 = GradientBoostingRegressor(random_state=1, n_estimators=10)
reg2 = RandomForestRegressor(random_state=1, n_estimators=10)
reg3 = LinearRegression()
ereg = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2), ('lr', reg3)])
ereg = ereg.fit(X_train, y_train)
print(ereg.score(X_test, y_test))


-0.052349220401247454
